In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/bank-customer-churn-modeling/Churn_Modelling.csv")
df1 = df.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [ ]:
for columns in df1:
  print(f"{columns}: {df1[columns].unique()}")

In [ ]:
exited_yes= df1[df1.Exited == 1].Geography
exited_no = df1[df1.Exited == 0].Geography

plt.hist([exited_yes,exited_no],label=("Yes","No"))
plt.xlabel("Country")
plt.ylabel("Number of People")
plt.legend()

In [ ]:
tenure_yes = df1[df1.Exited==1].Tenure
tenure_no = df1[df1.Exited==0].Tenure

plt.hist([tenure_yes,tenure_no],label=("Yes","No"))
plt.xlabel("Tenure")
plt.ylabel("Numper of People")
plt.legend()

In [ ]:
sns.countplot(data=df1,x=df1.Exited,hue=df1.Gender)

In [ ]:
sns.countplot(x=df1.IsActiveMember,hue=df1.Exited)

In [ ]:
sns.countplot(x=df1.HasCrCard,hue=df1.Exited)


In [ ]:
df1.dtypes

In [ ]:
cat_data = ["Geography","Gender"]
num_data = ["CreditScore","Age","EstimatedSalary","NumOfProducts","Balance","Tenure"]

df2 = pd.get_dummies(data=df1,columns=cat_data)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

df2[num_data] = scaler.fit_transform(df2[num_data])
df2.head()

In [ ]:
X = df2.drop("Exited",axis=1)
y = df2.Exited
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
model = keras.Sequential([keras.layers.Dense(100,input_shape=(13,),activation="relu"),
                          keras.layers.Dropout(0.5),
                          keras.layers.Dense(70,activation="relu"),
                          keras.layers.Dropout(0.5),
                          keras.layers.Dense(35,activation="relu"),
                          keras.layers.Dropout(0.5),
                          keras.layers.Dense(30,activation="relu"),
                          keras.layers.Dropout(0.5),
                          keras.layers.Dense(15,activation="relu"),
                          keras.layers.Dense(1,activation="sigmoid")



])
model.compile(loss="binary_crossentropy",metrics=["accuracy"],optimizer="adam")
early_stop = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)

In [ ]:
history= model.fit(X_train,y_train,epochs=100,callbacks=[early_stop])

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
history_df = pd.DataFrame(history.history)
plt.plot(history_df.accuracy,label="Accuracy")
#plt.plot(history_df.val_accuracy,label="Validation Accuracy")
plt.grid(True)
plt.legend()

In [ ]:
plt.plot(history_df.loss,label="Loss")
#plt.plot(history_df.val_loss,label="Val_Loss")
plt.grid(True)
plt.legend()

In [ ]:
prediction = model.predict(X_test)
prediction_binary = []

for i in prediction:
  if i >0.5:
    prediction_binary.append(1)
  else:
    prediction_binary.append(0)


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(prediction_binary,y_test))

In [ ]:
cm = confusion_matrix(prediction_binary,y_test)
sns.heatmap(cm,annot=True,fmt="d",cmap="YlGnBu")